In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
import feature_manager as fma
import classifier.multi_dnn_classifier as dnn
from random import randint
from keras import callbacks, losses
import visualizer
from keras.utils import np_utils
from sklearn.metrics import classification_report, confusion_matrix
import importlib
import tr_utils

In [ ]:
importlib.reload(fma)

In [ ]:
metric_list = [
    "loss",
    "accuracy",
    "precision",
    "recall",
    "precision-0.65",
    "recall-0.65",
    "precision-0.80",
    "recall-0.80",
    "precision-0.95",
    "recall-0.95"
]

symbol = "BTCUSDT"
trade_tf = "1d"
granular_tf = "1m"

tp = 0.1
sl = 0.08
md = 12

fm = fma.FeatureManager(
    target_col="trade_signal"
)

fm.import_data(symbol=symbol,trade_timeframe=trade_tf,granular_timeframe=granular_tf)

fm.prepare_trade_forward_data(
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
)

In [32]:
fm.build_features(lags=90)

Calculating features...

Adding feature: returns, dir, sma, boll, boll7, boll14, boll21, min, min7, min14, min21, max, max7, max14, max21, mom, mom7, mom14, mom21, vol, vol7, vol14, vol21, obv, mfi7, mfi14, mfi21, rsi7, rsi14, rsi21, adx7, adx14, adx21, roc, roc7, roc14, roc21, atr7, atr14, atr21, bop, ad, adosc, trange, ado, willr7, willr14, willr21, dx7, dx14, dx21, trix, ultosc, high, low, 

Normalizing feature: returns_lag_1, returns_lag_2, returns_lag_3, returns_lag_4, returns_lag_5, returns_lag_6, returns_lag_7, returns_lag_8, returns_lag_9, returns_lag_10, returns_lag_11, returns_lag_12, returns_lag_13, returns_lag_14, returns_lag_15, returns_lag_16, returns_lag_17, returns_lag_18, returns_lag_19, returns_lag_20, returns_lag_21, returns_lag_22, returns_lag_23, returns_lag_24, returns_lag_25, returns_lag_26, returns_lag_27, returns_lag_28, returns_lag_29, returns_lag_30, returns_lag_31, returns_lag_32, returns_lag_33, returns_lag_34, returns_lag_35, returns_lag_36, returns_lag_37

In [ ]:
importlib.reload(dnn)

In [33]:
results_list = []
results = dnn.evalute_classifier_k_folds(
    hu=100,
    fm=fm,
    fold_number=3,
    batch_size=20,
    set_class_weight=False,
    save_check_point=False,
    early_stopping=True,
    dropout=True,
    dropout_rate=0.3,
    gpu=False,
    metrics=metric_list,
    write_to_file=True)
results_list.append(results)


DATA PREPARATION PARAMS:
trade_timeframe:1d
take_profit_rate:0.1
stop_loss_rate:0.08
max_duration:12
lags:90
fold_number:3
train_size:0.7
val_size:0.15
categorical_label:True
rebalance:None
Splitting the data...
DATA:
Proccesing fold 0

FOLD 1
Train: 353, Validation: 75, Test: 77

Train:
Label 0: 136(38.53%)
Label 1: 122(34.56%)
Label 2: 95(26.91%)

Validation:
Label 0: 28(37.33%)
Label 1: 19(25.33%)
Label 2: 28(37.33%)

Test:
Label 0: 21(27.27%)
Label 1: 23(29.87%)
Label 2: 33(42.86%)
Proccesing fold 1

FOLD 2
Train: 353, Validation: 75, Test: 77

Train:
Label 0: 128(36.26%)
Label 1: 171(48.44%)
Label 2: 54(15.3%)

Validation:
Label 0: 16(21.33%)
Label 1: 19(25.33%)
Label 2: 40(53.33%)

Test:
Label 0: 14(18.18%)
Label 1: 34(44.16%)
Label 2: 29(37.66%)
Proccesing fold 2

FOLD 3
Train: 353, Validation: 75, Test: 77

Train:
Label 0: 104(29.46%)
Label 1: 105(29.75%)
Label 2: 144(40.79%)

Validation:
Label 0: 43(57.33%)
Label 1: 14(18.67%)
Label 2: 18(24.0%)

Test:
Label 0: 58(75.32%)
Lab

In [ ]:
metric_list = [
    "loss",
    "accuracy",
    "precision",
    "recall",
    "precision-0.65",
    "recall-0.65",
    "precision-0.80",
    "recall-0.80",
    "precision-0.95",
    "recall-0.95"
]

In [ ]:
results_list = []
results = dnn.evaluate_classifier(
    hu=100,
    fm=fm,
    laps=1,
    batch_size=20,
    set_class_weight=False,
    save_check_point=False,
    early_stopping=True,
    shuffle_before_split=False,
    dropout=True,
    dropout_rate=0.3,
    gpu=False,
    metrics=metric_list,
    write_to_file=True)
results_list.append(results)

In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df.to_csv("../out/evaluate.csv")

In [ ]:
classifier = dnn.MultiDNNClassifer()

dataset = classifier.prepare_data(
    data = fm.df,
    cols = fm.cols,
    shuffle_before_split= False,
    categorical_label=True,
    target_col="trade_signal"
)

initial_bias = tr_utils.init_imbalanced_bias(
    y_train=dataset[1]
)

classifier.configure(
    hu = 100, 
    dropout=True, 
    dropout_rate = 0.3,
    input_dim=len(fm.cols),
    output_bias=initial_bias,
    class_num=3
)